# Imports

In [ ]:
!pip install flair
!pip install sentence-transformers

In [2]:
import torch

import random
import os
import json
import re

from google.colab import drive, files

import pandas as pd

from flair.data import Sentence, Token, Label, Corpus
from flair.datasets import ColumnCorpus, ClassificationCorpus
from flair.embeddings import TransformerEmbeddings, TransformerDocumentEmbeddings, TransformerWordEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer

# Create the dataset samples

## Files setup

In [ ]:
# Note: This may not be available forever
!gdown 10V0fqO5aOk7yFMypH3884UVw0sQ7gday
!unzip -o -q rumoureval2019.zip
%cd rumoureval2019
!unzip -o -q rumoureval-2019-test-data.zip
!unzip -o -q rumoureval-2019-training-data.zip

The files are structured so that each folder inside reddit/twitter train/dev/test parent folder corresponds to a single conversation.

Each conversation contains a `replies` folder containing the raw tweet/reddit comments and a `structure.json` file indicating the nesting of the replies (a reply can either refer to the main comment or to another reply)

The functions below navigate the structure of the reddit and twitter folders and create a list of samples, which are `dict`s containing the data for a single conversation:

`id`: ID of the conversation

`title`: Main tweet / reddit post title

`structure`:`dict` containing the reply structure, based on their IDs

`replies`: `dict` containing the reply text contents, based on their IDs

We additionally perform a "cleaning" step that replaces URL and direct mentions (e.g. @username) by generic values, namely \$URL\$ and \$MENTION\$, on both titles and comments. This slightly improves the classification as it prevents the model from learning from the contents of usernames, and reduces noise caused by them and the URL tokens.

In [ ]:
def get_dirs(root):
  return [d for d in os.listdir(root) if os.path.isdir(os.path.join(root, d))]


def get_json_files(root):
  return [os.path.join(root, file) for file in os.listdir(root) if file.endswith('.json')]


def read_json_file(path):
  return json.load(open(path, 'r'))


def get_file_id(path):
  filename_without_extension = os.path.splitext(os.path.basename(path))[0]
  basename = filename_without_extension.split('/')[-1]
  return basename


# Clean twitter and reddit comments by replacing URLs with $URL$ and mentions
# with $MENTION$
# Adapted from https://github.com/Anushka-Prakash/RumourEval-2019-Stance-Detection
def clean_comment(comment):
  comment = re.sub(r"(http|ftp|https)\S+", "$URL$", comment.strip())
  comment = re.sub(r"(@[A-Za-z0-9_\\-]+)", "$MENTION$", comment.strip()) # Included '_' and '-' characters

  return comment


def create_sample_for_conversation(data_dir, conversation_dir, clean_comments):
  sample = dict()

  conversation_dir = os.path.join(data_dir, conversation_dir)
  source_json_files = get_json_files(conversation_dir+"/source-tweet/")
  reply_json_files = get_json_files(conversation_dir+"/replies/")

  source = read_json_file(source_json_files[0])

  post_title = None

  # The reddit and twitter reply files contain small differences in their structure,
  # so we have to distinguish them
  if "reddit" in data_dir: # It's a reddit post
    for child in source["data"]["children"]:
      if "title" in child["data"]:
        post_title = clean_comment(child["data"]["title"])
        break

  else: # It's a twitter thread
    post_title = clean_comment(source["text"])

  sample["id"] = get_file_id(source_json_files[0])
  sample["title"] = post_title
  sample["structure"] = read_json_file(conversation_dir+"/structure.json")
  sample["replies"] = dict()

  if "reddit" in data_dir: # It's a reddit post
    for reply in reply_json_files:
      reply_dict = read_json_file(reply)

      comment = None
      if "body" in reply_dict["data"]:
        comment = reply_dict["data"]["body"]
      # It's a wrongly parsed comment. Since there are not many, instead of
      # discarding the whole sample we simply put a "spurious" value
      if comment is None:
        comment = "invalid"

      if clean_comments:
        comment = clean_comment(comment)

      sample["replies"][get_file_id(reply)] = comment

  else: # It's a twitter thread
    for reply in reply_json_files:
      reply_dict = read_json_file(reply)

      # Get the file's path without the file extension
      sample["replies"][get_file_id(reply)] = clean_comment(reply_dict["text"])

  return sample


def create_samples(root, clean_comments=True):
  samples = []

  for data_dir in get_dirs(root):
    print(f"Reading conversations in {data_dir}...")
    data_dir = os.path.join(root, data_dir)
    conversation_dirs = get_dirs(data_dir)

    if "reddit" in data_dir: # It's a reddit conversations dir
      for i, conversation_dir in enumerate(conversation_dirs):
        samples.append(create_sample_for_conversation(data_dir, conversation_dir, clean_comments))

    else: # It's a twitter conversations dir
      for conversations_dir in get_dirs(data_dir): # We have another layer of subfolders
        conversations_dir = os.path.join(data_dir, conversations_dir)
        conversation_subdirs = get_dirs(conversations_dir)

        for i, conversation_dir in enumerate(conversation_subdirs):
          samples.append(create_sample_for_conversation(conversations_dir, conversation_dir, clean_comments))


  return samples

In [ ]:
samples_train = create_samples("rumoureval-2019-training-data")
samples_test = create_samples("rumoureval-2019-test-data")

# The ground truth files don't require any special processing, since we work with file IDs too
ground_truths_train = read_json_file("rumoureval-2019-training-data/train-key.json")["subtaskaenglish"]
ground_truths_dev = read_json_file("rumoureval-2019-training-data/dev-key.json")["subtaskaenglish"]
ground_truths_test = read_json_file("final-eval-key.json")["subtaskaenglish"]

# The twitter samples are not separated into train and development, so we will do
# it manually when training the BERT model
ground_truths = ground_truths_train | ground_truths_dev | ground_truths_test

Reading conversations in twitter-english...
Reading conversations in reddit-training-data...
Reading conversations in reddit-dev-data...
Reading conversations in reddit-test-data...
Reading conversations in twitter-en-test-data...


# Train flair's model (BERT models)

Now that we have parsed all samples, we can create the actual training/dev/test samples for the models following different strategies. A basic sample consists of a reply's text, to which we can add:


*   The main twitter thread's comment/reddit post's title
*   The comment's text the current sample is actually replying to (if it is the main one, we don't add it in order to avoid duplicates)
*   The depth of the reply with regard to the whole comment chain (1 if answering to the main comment, 2 if answering to a reply of the main comment, etc.)


These are all concatenated to the reply's text via '|' characters, which the BERT model should associate (by fine-tuning it) to a custom sentence separator. We don't use [SEP] since the model has been trained to only expect one [SEP] token, and we can have up to three separators.

A final sample would look like this, where the first three fields are optional:

`reply_depth | post_title | previous_comment | reply`



We additionally downsample the training dataset, since we have noticed that the `comment` class is extremely oversampled and can easily lead the model to only learn how to classify them


We have used the flair library for creating a training/dev/test dataset and training a BERT model for text classification (https://flairnlp.github.io/). This type of classification consists in using a [CLS] token, internally appended to the leftmost part of every sample, as sentence embeddings.

By doing this, every sample will have a single sentence embedding regardless of its contents or length, which will be then fed into a linear layer and classified via Softmax into one of the 4 classes (comment, support, deny or query)



In [ ]:
# Recursively navigates replies_structure and adds samples to bert_samples,
# considering the provided configuration (add the title or not, etc.)
def add_to_sample(sample, replies_structure, post_title, previous_comment, bert_samples, id, ground_truths, add_previous_comment, add_post_title, add_depth, depth):
  if replies_structure != []: # Stop condition
    for id, children in replies_structure.items():
      if id in sample["replies"]:
        sentence_text = sample["replies"][id]

        if add_previous_comment and previous_comment is not None:
          sentence_text = previous_comment + " | " + sentence_text

        # Do not add duplicate post_title | previous_comment | reply text when
        # post_title == previous_comment (it's a reply to the main post)
        if add_post_title and post_title is not None and previous_comment != post_title:
          sentence_text = post_title + " | " + sentence_text

        if add_depth:
          sentence_text = str(depth) + " | " + sentence_text # Changed from [SEP]

        sentence = Sentence(sentence_text)

        # Add its class as a label
        sentence.add_label("class", ground_truths[id])
        bert_samples.append(sentence)

        add_to_sample(sample, children, post_title, sample["replies"][id], bert_samples, id, ground_truths, add_previous_comment, add_post_title, add_depth, depth+1)


def create_bert_samples(samples, ground_truths, add_previous_comment, add_post_title, add_depth):
  bert_samples = []
  for sample in samples:
    title = sample["title"]
    structure = sample["structure"]

    for title_id, replies_structure in structure.items(): # We will only iterate once, it's the root (title)
      # We signal the id as None as we won't find it in the ground truth (we classify replies,
      # not the "root" post/comment)
      add_to_sample(sample, replies_structure, title, title, bert_samples, None, ground_truths, add_previous_comment, add_post_title, add_depth, 1)

  return bert_samples

In [ ]:
# Simple downsampling strategy implementation that limits classes to only
# oversample the least represented one by a factor of 1.25
def downsample(sentences):
  class_counts = dict()
  for sentence in sentences:
      class_val = sentence.get_label("class").value
      if class_val not in class_counts:
        class_counts[class_val] = 0
      class_counts[class_val] += 1

  min_samples = int(min(class_counts.values()) * 1.25) # Allow classes to be oversampled by a factor of 1.25

  balanced_sentences = []
  for class_label, count in class_counts.items():
      class_sentences = [sentence for sentence in sentences if sentence.get_label("class").value == class_label][:min_samples]
      balanced_sentences.extend(class_sentences)

  random.shuffle(balanced_sentences)

  return balanced_sentences

In [ ]:
# Converts the BERT samples to a fasttext format file (note: this is a flair requirement,
# it simply uses this format for handling datasets)
def to_fasttext_dataset(sentences, dest_name):
  with open(dest_name, "w", encoding="utf-8") as file:
    for sentence in sentences:
      line = f"__label__{sentence.get_label('class').value} {sentence.text.replace(',', ' ')}\n"
      file.write(line)

In [ ]:
# Creates train/dev/test datasets readable by flair based on the given configuration and
# fine-tunes a BERT model (indicated by model_name) for text classification
def test_model_configuration(samples_train, ground_truths, add_previous_comment, add_post_title, add_depth, model_name):
  # Create the BERT samples
  bert_samples_train = create_bert_samples(samples_train, ground_truths, add_previous_comment, add_post_title, add_depth)
  bert_samples_test = create_bert_samples(samples_test, ground_truths, add_previous_comment, add_post_title, add_depth)

  print("Sample example:", bert_samples_train[42])

  # Further split train into dev+train
  # The dev dataset will be used to
  random.shuffle(bert_samples_train)
  split_ratio = 0.8

  split_index = int(len(bert_samples_train) * split_ratio)

  bert_samples_dev = bert_samples_train[split_index:]
  bert_samples_train = bert_samples_train[:split_index]

  # Downsample the train dataset (the comment class is oversampled)
  bert_samples_train_downsampled = downsample(bert_samples_train)

  # Write the datasets in fasttext sample format
  to_fasttext_dataset(bert_samples_train_downsampled, "train_fasttext_format.txt")
  to_fasttext_dataset(bert_samples_dev, "dev_fasttext_format.txt")
  to_fasttext_dataset(bert_samples_test, "test_fasttext_format.txt")

  # Create flair's corpus
  data_folder = '.'
  corpus: Corpus = ClassificationCorpus(data_folder,
                                        test_file='test_fasttext_format.txt',
                                        dev_file='dev_fasttext_format.txt',
                                        train_file='train_fasttext_format.txt',
                                        label_type='class')

  # We can easily inspect the corpus structure (class to samples ratios...)
  print("Corpus statistics:\n", corpus.obtain_statistics())

  # Declare the embeddings we want to use
  document_embeddings = TransformerDocumentEmbeddings(model_name,
                                                      # Extract embeddings directly from the last layer
                                                      layers="-1",
                                                      # Indicate flair that we want to do [CLS] pooling, which is the strategy we want to do
                                                      cls_pooling="cls",
                                                      # We also want to finetune the model's embeddings, so that, at the very least, it learns
                                                      # the meaning of the '|' symbol we use as a comment separator
                                                      fine_tune=True,
                                                      # Hint that we only want to use the [CLS] token, so that we obtain a single sentence
                                                      # embedding per sample
                                                      is_token_embedding=False)

  example_sentence = bert_samples_train_downsampled[0]
  document_embeddings.embed(example_sentence)
  print(f"Example of an embedding:\n\t{example_sentence}\n\t{example_sentence.embedding}")

  # Instantiate flair's classifier for text classification (internally, it uses torch classifier layers)
  classifier = TextClassifier(document_embeddings,
                              label_dictionary=corpus.make_label_dictionary(label_type='class'),
                              label_type='class')

  # And train it
  trainer = ModelTrainer(classifier, corpus)
  trainer.fine_tune('resources/taggers/rumoureval_transformers',
                    learning_rate=5.0e-5,
                    mini_batch_size=4,
                    # We only need to train for a limited number of epochs
                    max_epochs=10)

  with torch.no_grad():
    torch.cuda.empty_cache()

Below are the results for every meaningful parameter configuration. We have used deberta-v3-base as our BERT model, which is one of the most recent/state of the art BERT models, from 2021.

Since there is a class imbalance (comment has, by far, the most samples), we need to choose the best model based on the F-macro metric, which calculates the F1-score on a per-class basis and then averages them (the F1-micro score, instead, simply calculates it for all samples).

The best performing configuration is the one that adds the title's text and the comment's depth indicator.

In [ ]:
add_previous_comment = False
add_post_title = False
add_depth = False

test_model_configuration(samples_train, ground_truths, add_previous_comment, add_post_title, add_depth, 'microsoft/deberta-v3-base')

Sample example: Sentence[27]: "“$MENTION$: The day #Ferguson cops told a dirty, bloody lie (via $MENTION$): $URL$ $URL$" → support (1.0)
2023-12-21 10:52:14,196 Reading data from .
2023-12-21 10:52:14,200 Train: train_fasttext_format.txt
2023-12-21 10:52:14,202 Dev: dev_fasttext_format.txt
2023-12-21 10:52:14,205 Test: test_fasttext_format.txt
2023-12-21 10:52:14,343 Initialized corpus . (label type name is 'class')
Corpus statistics:
 {
    "TRAIN": {
        "dataset": "TRAIN",
        "total_number_of_documents": 1643,
        "number_of_documents_per_class": {
            "query": 394,
            "support": 446,
            "comment": 446,
            "deny": 357
        },
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 36765,
            "min": 1,
            "max": 749,
            "avg": 22.376749847839317
        }
    },
    "TEST": {
        "dataset": "TEST",
        "total_number_of_documents": 1746,
        "number_of_documents_

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Example of an embedding:
	Sentence[6]: "$MENTION$ which version?" → query (1.0)
	tensor([ 1.5943e-01,  2.5077e-01,  5.7776e-02, -9.0526e-02,  3.7470e-02,
        -1.3958e-01,  5.3581e-03,  1.5695e-01, -2.5912e-01,  6.8150e-02,
         5.1543e-02, -2.9987e-01, -7.9545e-02,  2.5842e-01,  1.4114e-01,
         7.3076e-03, -1.3795e-01, -4.8675e-02, -1.6805e-01,  1.3327e-02,
        -2.2311e-01, -1.9917e-01, -5.7200e-02, -2.4599e-03,  8.1132e-03,
        -4.5756e-02, -1.1947e-01, -4.5453e-02,  8.2243e-02,  1.0843e-01,
        -1.8072e-01, -8.1738e-03, -9.5604e-02, -7.4721e-02,  2.2969e-01,
         1.7892e-01, -8.2302e-02,  4.8289e-02, -1.3907e-01,  2.5242e-02,
        -2.0426e-01, -5.1149e-02,  1.0878e-02, -5.4616e-02,  7.5109e-02,
         1.2400e-02,  6.8857e-02, -5.9202e-02, -1.1723e-01,  6.0641e-02,
        -1.3637e-01, -1.4378e-01,  1.3940e-01, -1.1313e-01, -1.3165e+00,
         7.0044e-02, -2.8430e-01,  1.3835e-01,  1.3929e-01, -2.3284e-01,
         1.1076e-01, -1.8614e-01,  4.6364e-

0it [00:00, ?it/s]
1643it [00:00, 2766.51it/s]

2023-12-21 10:52:25,045 Dictionary created for label 'class' with 4 values: support (seen 446 times), comment (seen 446 times), query (seen 394 times), deny (seen 357 times)


...

100%|██████████| 80/80 [00:14<00:00,  5.39it/s]

2023-12-21 11:05:43,280 DEV : loss 5.071529865264893 - f1-score (micro avg)  0.4905


2023-12-21 11:05:49,129 ----------------------------------------------------------------------------------------------------
2023-12-21 11:05:49,132 Testing using last state of model ...


100%|██████████| 110/110 [00:17<00:00,  6.44it/s]

2023-12-21 11:06:06,254 
Results:
- F-score (micro) 0.579
- F-score (macro) 0.4189
- Accuracy 0.579

By class:
              precision    recall  f1-score   support

     comment     0.9248    0.5752    0.7093      1476
     support     0.1340    0.4808    0.2096       104
        deny     0.2089    0.6100    0.3112       100
       query     0.3129    0.7727    0.4454        66

    accuracy                         0.5790      1746
   macro avg     0.3952    0.6097    0.4189      1746
weighted avg     0.8136    0.5790    0.6467      1746

2023-12-21 11:06:06,257 ----------------------------------------------------------------------------------------------------


In [ ]:
add_previous_comment = True
add_post_title = False
add_depth = False

test_model_configuration(samples_train, ground_truths, add_previous_comment, add_post_title, add_depth, 'microsoft/deberta-v3-base')

Sample example: Sentence[51]: "The day #Ferguson cops told a dirty, bloody lie (via $MENTION$): $URL$ $URL$ | “$MENTION$: The day #Ferguson cops told a dirty, bloody lie (via $MENTION$): $URL$ $URL$" → support (1.0)
2023-12-21 11:39:18,401 Reading data from .
2023-12-21 11:39:18,402 Train: train_fasttext_format.txt
2023-12-21 11:39:18,407 Dev: dev_fasttext_format.txt
2023-12-21 11:39:18,408 Test: test_fasttext_format.txt
2023-12-21 11:39:18,499 Initialized corpus . (label type name is 'class')
Corpus statistics:
 {
    "TRAIN": {
        "dataset": "TRAIN",
        "total_number_of_documents": 1586,
        "number_of_documents_per_class": {
            "support": 427,
            "query": 390,
            "deny": 342,
            "comment": 427
        },
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 75371,
            "min": 11,
            "max": 573,
            "avg": 47.52269861286255
        }
    },
    "TEST": {
        "dataset": "

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Example of an embedding:
	Sentence[37]: "$MENTION$ $MENTION$ Added to what happened in Quebec on Monday, the people who protect our nations need our full support. | $MENTION$ $MENTION$ Yes they do." → support (1.0)
	tensor([ 1.4228e-01,  2.2221e-01,  6.3073e-02, -5.9620e-02,  8.1782e-02,
        -1.2725e-01, -1.6231e-03,  9.6102e-02, -1.7054e-01,  1.0007e-01,
         5.2429e-02, -2.7547e-01,  4.2257e-02,  2.3015e-01,  7.9419e-02,
        -4.4294e-02, -1.2677e-01, -9.0077e-03, -1.3385e-01, -4.0198e-02,
        -2.2864e-01, -1.8995e-01, -2.7270e-02,  5.1124e-02, -2.6129e-03,
        -2.8032e-02, -4.0835e-02, -5.9706e-02,  2.9471e-02,  1.0979e-01,
        -8.9512e-02, -2.0280e-02, -6.7833e-02, -4.4873e-02,  1.2313e-01,
         1.2662e-01, -7.3993e-02,  4.2909e-02, -1.1559e-01,  6.7592e-02,
        -1.6930e-01, -1.9147e-02,  1.7154e-02, -4.2533e-02,  7.8238e-02,
         5.9729e-03,  9.0034e-02, -1.2926e-01, -3.4793e-02,  1.7157e-02,
        -5.1485e-02, -1.2702e-01,  9.5652e-02, -9.5657

0it [00:00, ?it/s]
1586it [00:01, 802.34it/s]

2023-12-21 11:39:37,293 Dictionary created for label 'class' with 4 values: support (seen 427 times), comment (seen 427 times), query (seen 390 times), deny (seen 342 times)
2023-12-21 11:39:37,305 ----------------------------------------------------------------------------------------------------
2023-12-21 11:39:37,311 Model: "TextClassifier(
  (embeddings): TransformerDocumentEmbeddings(
    (model): DebertaV2Model(
      (embeddings): DebertaV2Embeddings(
        (word_embeddings): Embedding(128101, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
        (dropout): StableDropout()
      )
      (encoder): DebertaV2Encoder(
        (layer): ModuleList(
          (0-11): 12 x DebertaV2Layer(
            (attention): DebertaV2Attention(
              (self): DisentangledSelfAttention(
                (query_proj): Linear(in_features=768, out_features=768, bias=True)
                (key_proj): Linear(in_features=768, out_features=768, bias=True)
       

...

100%|██████████| 80/80 [00:22<00:00,  3.51it/s]

2023-12-21 11:55:01,184 DEV : loss 4.791125297546387 - f1-score (micro avg)  0.5079


2023-12-21 11:55:08,997 ----------------------------------------------------------------------------------------------------
2023-12-21 11:55:09,002 Testing using last state of model ...


100%|██████████| 110/110 [00:26<00:00,  4.13it/s]

2023-12-21 11:55:35,702 
Results:
- F-score (micro) 0.5624
- F-score (macro) 0.406
- Accuracy 0.5624

By class:
              precision    recall  f1-score   support

     comment     0.9231    0.5610    0.6979      1476
     support     0.1117    0.4038    0.1750       104
        deny     0.1963    0.6400    0.3005       100
       query     0.3265    0.7273    0.4507        66

    accuracy                         0.5624      1746
   macro avg     0.3894    0.5830    0.4060      1746
weighted avg     0.8106    0.5624    0.6346      1746

2023-12-21 11:55:35,703 ----------------------------------------------------------------------------------------------------


In [ ]:
# microsoft/deberta-v3-base
add_previous_comment = False
add_post_title = True
add_depth = False

test_model_configuration(samples_train, ground_truths, add_previous_comment, add_post_title, add_depth, 'microsoft/deberta-v3-base')

Sample example: Sentence[27]: "“$MENTION$: The day #Ferguson cops told a dirty, bloody lie (via $MENTION$): $URL$ $URL$" → support (1.0)
2023-12-21 11:56:37,613 Reading data from .
2023-12-21 11:56:37,618 Train: train_fasttext_format.txt
2023-12-21 11:56:37,621 Dev: dev_fasttext_format.txt
2023-12-21 11:56:37,623 Test: test_fasttext_format.txt
2023-12-21 11:56:37,787 Initialized corpus . (label type name is 'class')
Corpus statistics:
 {
    "TRAIN": {
        "dataset": "TRAIN",
        "total_number_of_documents": 1577,
        "number_of_documents_per_class": {
            "support": 428,
            "comment": 428,
            "deny": 343,
            "query": 378
        },
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 47357,
            "min": 1,
            "max": 414,
            "avg": 30.029803424223207
        }
    },
    "TEST": {
        "dataset": "TEST",
        "total_number_of_documents": 1746,
        "number_of_documents_

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Example of an embedding:
	Sentence[24]: "#Australia #ISIS #Terrorism $MENTION$: #BREAKING #Hostages held inside #Sydney cafe, Islamic flag held up" → support (1.0)
	tensor([ 1.5658e-01,  2.2798e-01,  2.9953e-02, -6.3864e-02,  7.7165e-02,
        -1.2456e-01,  5.5820e-04,  1.2335e-01, -2.1314e-01,  7.1097e-02,
         7.5221e-02, -2.9232e-01,  4.9414e-04,  2.8242e-01,  8.7127e-02,
        -1.9393e-02, -1.1177e-01, -3.4503e-02, -1.2788e-01, -1.1266e-03,
        -2.5108e-01, -1.8039e-01, -7.6450e-02,  1.5031e-02,  6.5897e-02,
        -4.2171e-02, -4.3525e-02, -6.9999e-02,  5.2235e-02,  1.0993e-01,
        -1.5200e-01,  7.1379e-03, -1.1858e-01, -4.7109e-02,  1.8609e-01,
         1.6253e-01, -1.1561e-01,  9.1119e-02, -1.2881e-01,  5.2922e-02,
        -1.9519e-01,  6.6728e-03,  1.8223e-02, -7.5289e-02,  7.1455e-02,
         8.1270e-03,  6.4313e-02, -1.0381e-01, -3.4297e-02,  1.5270e-02,
        -6.5520e-02, -1.4845e-01,  1.0706e-01, -8.6872e-02, -1.1260e+00,
         7.8040e-02, -2.6892e-01

0it [00:00, ?it/s]
1577it [00:00, 2178.40it/s]

2023-12-21 11:56:50,292 Dictionary created for label 'class' with 4 values: support (seen 428 times), comment (seen 428 times), query (seen 378 times), deny (seen 343 times)
2023-12-21 11:56:50,302 ----------------------------------------------------------------------------------------------------
2023-12-21 11:56:50,306 Model: "TextClassifier(
  (embeddings): TransformerDocumentEmbeddings(
    (model): DebertaV2Model(
      (embeddings): DebertaV2Embeddings(
        (word_embeddings): Embedding(128101, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
        (dropout): StableDropout()
      )
      (encoder): DebertaV2Encoder(
        (layer): ModuleList(
          (0-11): 12 x DebertaV2Layer(
            (attention): DebertaV2Attention(
              (self): DisentangledSelfAttention(
                (query_proj): Linear(in_features=768, out_features=768, bias=True)
                (key_proj): Linear(in_features=768, out_features=768, bias=True)
       

...

100%|██████████| 80/80 [00:14<00:00,  5.55it/s]

2023-12-21 12:09:41,778 DEV : loss 3.924910545349121 - f1-score (micro avg)  0.5331


2023-12-21 12:09:47,167 ----------------------------------------------------------------------------------------------------
2023-12-21 12:09:47,175 Testing using last state of model ...


100%|██████████| 110/110 [00:18<00:00,  5.90it/s]


2023-12-21 12:10:05,872 
Results:
- F-score (micro) 0.5584
- F-score (macro) 0.4002
- Accuracy 0.5584

By class:
              precision    recall  f1-score   support

     comment     0.9169    0.5535    0.6903      1476
     support     0.1025    0.3558    0.1591       104
        deny     0.2250    0.7200    0.3429       100
       query     0.2816    0.7424    0.4083        66

    accuracy                         0.5584      1746
   macro avg     0.3815    0.5929    0.4002      1746
weighted avg     0.8048    0.5584    0.6281      1746

2023-12-21 12:10:05,874 ----------------------------------------------------------------------------------------------------


In [ ]:
# microsoft/deberta-v3-base
add_previous_comment = True
add_post_title = True
add_depth = False

test_model_configuration(samples_train, ground_truths, add_previous_comment, add_post_title, add_depth, 'microsoft/deberta-v3-base')

Sample example: Sentence[51]: "The day #Ferguson cops told a dirty, bloody lie (via $MENTION$): $URL$ $URL$ | “$MENTION$: The day #Ferguson cops told a dirty, bloody lie (via $MENTION$): $URL$ $URL$" → support (1.0)
2023-12-21 12:10:16,481 Reading data from .
2023-12-21 12:10:16,482 Train: train_fasttext_format.txt
2023-12-21 12:10:16,486 Dev: dev_fasttext_format.txt
2023-12-21 12:10:16,488 Test: test_fasttext_format.txt
2023-12-21 12:10:16,578 Initialized corpus . (label type name is 'class')
Corpus statistics:
 {
    "TRAIN": {
        "dataset": "TRAIN",
        "total_number_of_documents": 1647,
        "number_of_documents_per_class": {
            "deny": 360,
            "query": 387,
            "comment": 450,
            "support": 450
        },
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 92447,
            "min": 13,
            "max": 664,
            "avg": 56.13054037644201
        }
    },
    "TEST": {
        "dataset": "

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Example of an embedding:
	Sentence[53]: "AFP reports there are 2 dead, and 5 hostages being held in the Kosher store in Eastern Paris; separate incident to charlie hebdo shooters. | $MENTION$ $MENTION$ Officials have apparently confirmed that the deli is linked to #CharlieHebdo &amp; killing of officer." → deny (1.0)
	tensor([ 1.6262e-01,  2.2467e-01,  3.0557e-02, -8.8129e-02,  5.6375e-02,
        -1.2209e-01,  1.1113e-02,  1.1126e-01, -2.0146e-01,  8.9319e-02,
         9.0549e-02, -3.0501e-01,  1.3721e-02,  2.5434e-01,  8.2459e-02,
        -3.8001e-02, -1.3914e-01, -6.6079e-03, -1.2918e-01, -9.8285e-03,
        -2.0756e-01, -2.2062e-01, -3.1388e-02,  7.2702e-02, -1.5297e-02,
        -6.0766e-02, -3.9302e-02, -5.4088e-02,  2.4068e-02,  9.7093e-02,
        -9.1520e-02, -7.6734e-03, -6.7796e-02, -7.4935e-02,  1.4654e-01,
         1.6277e-01, -8.0084e-02,  5.2077e-02, -1.3061e-01,  6.9444e-02,
        -1.8608e-01, -1.0225e-02, -1.7062e-02, -7.3506e-02,  1.1273e-01,
         1.6067e-02,  5

0it [00:00, ?it/s]
1647it [00:02, 696.11it/s]

2023-12-21 12:10:36,865 Dictionary created for label 'class' with 4 values: comment (seen 450 times), support (seen 450 times), query (seen 387 times), deny (seen 360 times)
2023-12-21 12:10:36,874 ----------------------------------------------------------------------------------------------------
2023-12-21 12:10:36,878 Model: "TextClassifier(
  (embeddings): TransformerDocumentEmbeddings(
    (model): DebertaV2Model(
      (embeddings): DebertaV2Embeddings(
        (word_embeddings): Embedding(128101, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
        (dropout): StableDropout()
      )
      (encoder): DebertaV2Encoder(
        (layer): ModuleList(
          (0-11): 12 x DebertaV2Layer(
            (attention): DebertaV2Attention(
              (self): DisentangledSelfAttention(
                (query_proj): Linear(in_features=768, out_features=768, bias=True)
                (key_proj): Linear(in_features=768, out_features=768, bias=True)
       

...

2023-12-21 12:26:19,949 DEV : loss 5.21014404296875 - f1-score (micro avg)  0.4929
2023-12-21 12:26:21,078 ----------------------------------------------------------------------------------------------------
2023-12-21 12:26:27,949 epoch 10 - iter 41/412 - loss 0.01761200 - time (sec): 6.87 - samples/sec: 23.88 - lr: 0.000005 - momentum: 0.000000
2023-12-21 12:26:35,422 epoch 10 - iter 82/412 - loss 0.02906772 - time (sec): 14.34 - samples/sec: 22.87 - lr: 0.000005 - momentum: 0.000000
2023-12-21 12:26:42,397 epoch 10 - iter 123/412 - loss 0.02795712 - time (sec): 21.31 - samples/sec: 23.08 - lr: 0.000004 - momentum: 0.000000
2023-12-21 12:26:50,211 epoch 10 - iter 164/412 - loss 0.04748201 - time (sec): 29.13 - samples/sec: 22.52 - lr: 0.000003 - momentum: 0.000000
2023-12-21 12:26:58,719 epoch 10 - iter 205/412 - loss 0.04407823 - time (sec): 37.64 - samples/sec: 21.79 - lr: 0.000003 - momentum: 0.000000
2023-12-21 12:27:05,877 epoch 10 - iter 246/412 - loss 0.03741078 - time (sec): 

100%|██████████| 80/80 [00:25<00:00,  3.16it/s]


2023-12-21 12:28:03,195 DEV : loss 5.538442611694336 - f1-score (micro avg)  0.474
2023-12-21 12:28:08,904 ----------------------------------------------------------------------------------------------------
2023-12-21 12:28:08,908 Testing using last state of model ...


100%|██████████| 110/110 [00:27<00:00,  3.98it/s]

2023-12-21 12:28:36,578 
Results:
- F-score (micro) 0.516
- F-score (macro) 0.3804
- Accuracy 0.516

By class:
              precision    recall  f1-score   support

     comment     0.9072    0.5102    0.6531      1476
     support     0.0747    0.2788    0.1179       104
        deny     0.1867    0.7300    0.2974       100
       query     0.3358    0.6970    0.4532        66

    accuracy                         0.5160      1746
   macro avg     0.3761    0.5540    0.3804      1746
weighted avg     0.7948    0.5160    0.5933      1746

2023-12-21 12:28:36,579 ----------------------------------------------------------------------------------------------------


In [ ]:
# microsoft/deberta-v3-base
add_previous_comment = False
add_post_title = False
add_depth = True

test_model_configuration(samples_train, ground_truths, add_previous_comment, add_post_title, add_depth, 'microsoft/deberta-v3-base')

Sample example: Sentence[29]: "1 | “$MENTION$: The day #Ferguson cops told a dirty, bloody lie (via $MENTION$): $URL$ $URL$" → support (1.0)
2023-12-21 12:29:52,579 Reading data from .
2023-12-21 12:29:52,582 Train: train_fasttext_format.txt
2023-12-21 12:29:52,583 Dev: dev_fasttext_format.txt
2023-12-21 12:29:52,586 Test: test_fasttext_format.txt
2023-12-21 12:29:52,676 Initialized corpus . (label type name is 'class')
Corpus statistics:
 {
    "TRAIN": {
        "dataset": "TRAIN",
        "total_number_of_documents": 1619,
        "number_of_documents_per_class": {
            "support": 437,
            "comment": 437,
            "query": 395,
            "deny": 350
        },
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 40815,
            "min": 3,
            "max": 751,
            "avg": 25.210006176652254
        }
    },
    "TEST": {
        "dataset": "TEST",
        "total_number_of_documents": 1746,
        "number_of_docume

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Example of an embedding:
	Sentence[17]: "2 | $MENTION$ $MENTION$ Yes, but think it was too dangerous." → support (1.0)
	tensor([ 1.4270e-01,  2.5445e-01,  4.7761e-02, -8.6881e-02,  8.7037e-02,
        -1.2384e-01,  4.5833e-02,  1.4208e-01, -2.1360e-01,  9.1224e-02,
         4.8549e-02, -3.1168e-01, -1.2635e-02,  2.6830e-01,  9.6695e-02,
        -4.4871e-02, -1.0445e-01, -6.6423e-02, -1.7607e-01, -4.2733e-02,
        -2.0852e-01, -1.7605e-01, -4.3894e-02,  3.5927e-02,  3.7008e-02,
        -5.5675e-02, -4.6586e-02, -8.6087e-02,  3.9670e-02,  1.2012e-01,
        -1.5467e-01, -1.7173e-02, -1.2741e-01, -1.0395e-01,  1.4453e-01,
         1.7798e-01, -8.5988e-02,  4.4911e-02, -1.0264e-01,  3.7803e-02,
        -1.9328e-01, -4.2592e-02,  1.6599e-03, -4.7770e-02,  5.3300e-02,
         4.8131e-02,  5.8819e-02, -1.2751e-01, -2.2832e-02, -1.8337e-03,
        -5.9992e-02, -1.5252e-01,  1.5046e-01, -1.1368e-01, -9.1492e-01,
         8.1495e-02, -3.0327e-01,  1.6789e-01,  9.1057e-02, -2.0041e-01,
    

0it [00:00, ?it/s]
1619it [00:00, 2105.68it/s]

2023-12-21 12:30:03,085 Dictionary created for label 'class' with 4 values: support (seen 437 times), comment (seen 437 times), query (seen 395 times), deny (seen 350 times)
2023-12-21 12:30:03,095 ----------------------------------------------------------------------------------------------------
2023-12-21 12:30:03,098 Model: "TextClassifier(
  (embeddings): TransformerDocumentEmbeddings(
    (model): DebertaV2Model(
      (embeddings): DebertaV2Embeddings(
        (word_embeddings): Embedding(128101, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
        (dropout): StableDropout()
      )
      (encoder): DebertaV2Encoder(
        (layer): ModuleList(
          (0-11): 12 x DebertaV2Layer(
            (attention): DebertaV2Attention(
              (self): DisentangledSelfAttention(
                (query_proj): Linear(in_features=768, out_features=768, bias=True)
                (key_proj): Linear(in_features=768, out_features=768, bias=True)
       

...

100%|██████████| 80/80 [00:16<00:00,  4.97it/s]

2023-12-21 12:43:32,541 DEV : loss 5.205692291259766 - f1-score (micro avg)  0.4874


2023-12-21 12:43:39,134 ----------------------------------------------------------------------------------------------------
2023-12-21 12:43:39,139 Testing using last state of model ...


100%|██████████| 110/110 [00:17<00:00,  6.37it/s]

2023-12-21 12:43:56,468 
Results:
- F-score (micro) 0.6128
- F-score (macro) 0.4127
- Accuracy 0.6128

By class:
              precision    recall  f1-score   support

     comment     0.9269    0.6267    0.7478      1476
     support     0.1010    0.2788    0.1483       104
        deny     0.2334    0.6700    0.3463       100
       query     0.2816    0.7424    0.4083        66

    accuracy                         0.6128      1746
   macro avg     0.3857    0.5795    0.4127      1746
weighted avg     0.8136    0.6128    0.6762      1746

2023-12-21 12:43:56,474 ----------------------------------------------------------------------------------------------------


In [ ]:
# microsoft/deberta-v3-base
add_previous_comment = True
add_post_title = True
add_depth = True

test_model_configuration(samples_train, ground_truths, add_previous_comment, add_post_title, add_depth, 'microsoft/deberta-v3-base')

Sample example: Sentence[53]: "1 | The day #Ferguson cops told a dirty, bloody lie (via $MENTION$): $URL$ $URL$ | “$MENTION$: The day #Ferguson cops told a dirty, bloody lie (via $MENTION$): $URL$ $URL$" → support (1.0)
2023-12-21 12:44:06,702 Reading data from .
2023-12-21 12:44:06,704 Train: train_fasttext_format.txt
2023-12-21 12:44:06,706 Dev: dev_fasttext_format.txt
2023-12-21 12:44:06,708 Test: test_fasttext_format.txt
2023-12-21 12:44:06,801 Initialized corpus . (label type name is 'class')
Corpus statistics:
 {
    "TRAIN": {
        "dataset": "TRAIN",
        "total_number_of_documents": 1606,
        "number_of_documents_per_class": {
            "support": 433,
            "deny": 347,
            "query": 393,
            "comment": 433
        },
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 90466,
            "min": 16,
            "max": 601,
            "avg": 56.33001245330013
        }
    },
    "TEST": {
        "dataset

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Example of an embedding:
	Sentence[42]: "1 | FIRST PHOTOS: Hostage pressed against a window. #SydneySiege $URL$ | $MENTION$ CAUTION Be aware that this could be a #FalseFlag Event unfolding right across the street from channel 7 coincidence? Perhaps" → support (1.0)
	tensor([ 1.5862e-01,  2.0805e-01,  8.2993e-03, -8.3260e-02,  5.8025e-02,
        -1.2342e-01,  6.4949e-03,  1.2236e-01, -2.2791e-01,  9.8080e-02,
         7.2621e-02, -3.2411e-01,  1.2610e-02,  2.9934e-01,  8.7222e-02,
        -3.5200e-02, -1.3333e-01,  1.2582e-02, -1.5071e-01, -7.6307e-03,
        -2.3050e-01, -2.0185e-01, -5.6920e-02,  4.3676e-02,  1.1515e-02,
        -6.5593e-02, -2.6738e-02, -5.8312e-02,  5.3346e-02,  1.0934e-01,
        -1.0719e-01,  1.1189e-02, -1.0134e-01, -6.4594e-02,  1.5853e-01,
         1.6814e-01, -6.9279e-02,  5.9100e-02, -1.0106e-01,  7.1812e-02,
        -1.7794e-01, -1.4851e-02, -1.5957e-02, -7.0774e-02,  8.7063e-02,
         2.3831e-02,  6.6053e-02, -1.2024e-01, -2.6739e-02,  2.7363e-02,
   

0it [00:00, ?it/s]
1606it [00:02, 685.17it/s]

2023-12-21 12:44:27,901 Dictionary created for label 'class' with 4 values: support (seen 433 times), comment (seen 433 times), query (seen 393 times), deny (seen 347 times)
2023-12-21 12:44:27,914 ----------------------------------------------------------------------------------------------------
2023-12-21 12:44:27,919 Model: "TextClassifier(
  (embeddings): TransformerDocumentEmbeddings(
    (model): DebertaV2Model(
      (embeddings): DebertaV2Embeddings(
        (word_embeddings): Embedding(128101, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
        (dropout): StableDropout()
      )
      (encoder): DebertaV2Encoder(
        (layer): ModuleList(
          (0-11): 12 x DebertaV2Layer(
            (attention): DebertaV2Attention(
              (self): DisentangledSelfAttention(
                (query_proj): Linear(in_features=768, out_features=768, bias=True)
                (key_proj): Linear(in_features=768, out_features=768, bias=True)
       

...

100%|██████████| 80/80 [00:25<00:00,  3.10it/s]

2023-12-21 13:00:59,030 DEV : loss 3.290358781814575 - f1-score (micro avg)  0.4574


2023-12-21 13:01:04,621 ----------------------------------------------------------------------------------------------------
2023-12-21 13:01:04,625 Testing using last state of model ...


100%|██████████| 110/110 [00:28<00:00,  3.90it/s]

2023-12-21 13:01:32,859 
Results:
- F-score (micro) 0.4387
- F-score (macro) 0.3732
- Accuracy 0.4387

By class:
              precision    recall  f1-score   support

     comment     0.9376    0.4072    0.5678      1476
     support     0.0930    0.6346    0.1622       104
        deny     0.1970    0.5200    0.2857       100
       query     0.3588    0.7121    0.4772        66

    accuracy                         0.4387      1746
   macro avg     0.3966    0.5685    0.3732      1746
weighted avg     0.8230    0.4387    0.5240      1746

2023-12-21 13:01:32,861 ----------------------------------------------------------------------------------------------------


In [ ]:
# microsoft/deberta-v3-base
add_previous_comment = True
add_post_title = False
add_depth = True

test_model_configuration(samples_train, ground_truths, add_previous_comment, add_post_title, add_depth, 'microsoft/deberta-v3-base')

Sample example: Sentence[53]: "1 | The day #Ferguson cops told a dirty, bloody lie (via $MENTION$): $URL$ $URL$ | “$MENTION$: The day #Ferguson cops told a dirty, bloody lie (via $MENTION$): $URL$ $URL$" → support (1.0)
2023-12-21 13:13:58,795 Reading data from .
2023-12-21 13:13:58,798 Train: train_fasttext_format.txt
2023-12-21 13:13:58,799 Dev: dev_fasttext_format.txt
2023-12-21 13:13:58,803 Test: test_fasttext_format.txt
2023-12-21 13:13:58,953 Initialized corpus . (label type name is 'class')
Corpus statistics:
 {
    "TRAIN": {
        "dataset": "TRAIN",
        "total_number_of_documents": 1643,
        "number_of_documents_per_class": {
            "comment": 450,
            "query": 383,
            "deny": 360,
            "support": 450
        },
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 81420,
            "min": 13,
            "max": 722,
            "avg": 49.55569080949483
        }
    },
    "TEST": {
        "dataset

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Example of an embedding:
	Sentence[62]: "3 | $MENTION$ $MENTION$ $MENTION$ Why is he a hero? His life is worth nothing more than the life of a civilian would be... | $MENTION$ $MENTION$ $MENTION$ Because soldiers and police are always running towards what we are running away from! That's a hero!" → comment (1.0)
	tensor([ 1.3220e-01,  1.9850e-01,  7.3510e-02, -7.5041e-02,  8.0121e-02,
        -1.1836e-01, -1.2611e-02,  9.2433e-02, -1.5213e-01,  1.1440e-01,
         5.4930e-02, -2.6069e-01,  2.9575e-02,  2.2835e-01,  9.5882e-02,
        -4.8221e-02, -1.1460e-01,  1.8375e-02, -1.0880e-01, -1.4825e-02,
        -2.1163e-01, -2.0967e-01, -1.8485e-02,  4.5558e-02, -4.5602e-03,
        -1.4737e-02, -5.6363e-02, -4.6349e-02,  3.1586e-02,  1.2927e-01,
        -6.5931e-02,  9.6189e-03, -5.3213e-02, -3.0546e-02,  1.0771e-01,
         1.1329e-01, -5.7490e-02,  4.1111e-02, -1.1803e-01,  7.0540e-02,
        -1.9045e-01, -6.1328e-03,  1.8892e-02, -8.9352e-02,  1.0730e-01,
        -1.3585e-03,  8.6849

0it [00:00, ?it/s]
1643it [00:01, 1387.58it/s]

2023-12-21 13:14:25,123 Dictionary created for label 'class' with 4 values: comment (seen 450 times), support (seen 450 times), query (seen 383 times), deny (seen 360 times)
2023-12-21 13:14:25,133 ----------------------------------------------------------------------------------------------------
2023-12-21 13:14:25,135 Model: "TextClassifier(
  (embeddings): TransformerDocumentEmbeddings(
    (model): DebertaV2Model(
      (embeddings): DebertaV2Embeddings(
        (word_embeddings): Embedding(128101, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
        (dropout): StableDropout()
      )
      (encoder): DebertaV2Encoder(
        (layer): ModuleList(
          (0-11): 12 x DebertaV2Layer(
            (attention): DebertaV2Attention(
              (self): DisentangledSelfAttention(
                (query_proj): Linear(in_features=768, out_features=768, bias=True)
                (key_proj): Linear(in_features=768, out_features=768, bias=True)
       

...

100%|██████████| 80/80 [00:21<00:00,  3.65it/s]

2023-12-21 13:30:27,511 DEV : loss 4.063159465789795 - f1-score (micro avg)  0.5513


2023-12-21 13:30:32,361 ----------------------------------------------------------------------------------------------------
2023-12-21 13:30:32,367 Testing using last state of model ...


100%|██████████| 110/110 [00:25<00:00,  4.28it/s]

2023-12-21 13:30:58,126 
Results:
- F-score (micro) 0.6025
- F-score (macro) 0.3983
- Accuracy 0.6025

By class:
              precision    recall  f1-score   support

     comment     0.9284    0.6145    0.7395      1476
        deny     0.1972    0.5700    0.2931       100
     support     0.1362    0.3365    0.1939       104
       query     0.2377    0.8030    0.3668        66

    accuracy                         0.6025      1746
   macro avg     0.3749    0.5810    0.3983      1746
weighted avg     0.8132    0.6025    0.6673      1746

2023-12-21 13:30:58,127 ----------------------------------------------------------------------------------------------------


In [ ]:
# microsoft/deberta-v3-base
add_previous_comment = False
add_post_title = True
add_depth = True

test_model_configuration(samples_train, ground_truths, add_previous_comment, add_post_title, add_depth, 'microsoft/deberta-v3-base')

Sample example: Sentence[29]: "1 | “$MENTION$: The day #Ferguson cops told a dirty, bloody lie (via $MENTION$): $URL$ $URL$" → support (1.0)
2023-12-22 09:07:02,390 Reading data from .
2023-12-22 09:07:02,391 Train: train_fasttext_format.txt
2023-12-22 09:07:02,395 Dev: dev_fasttext_format.txt
2023-12-22 09:07:02,397 Test: test_fasttext_format.txt
2023-12-22 09:07:02,478 Initialized corpus . (label type name is 'class')
Corpus statistics:
 {
    "TRAIN": {
        "dataset": "TRAIN",
        "total_number_of_documents": 1594,
        "number_of_documents_per_class": {
            "support": 428,
            "query": 395,
            "deny": 343,
            "comment": 428
        },
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 53777,
            "min": 5,
            "max": 764,
            "avg": 33.737139272271015
        }
    },
    "TEST": {
        "dataset": "TEST",
        "total_number_of_documents": 1746,
        "number_of_docume

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


...

100%|██████████| 80/80 [00:12<00:00,  6.24it/s]

2023-12-22 09:20:15,474 DEV : loss 5.076405048370361 - f1-score (micro avg)  0.5473


2023-12-22 09:20:19,533 ----------------------------------------------------------------------------------------------------
2023-12-22 09:20:19,542 Testing using last state of model ...


100%|██████████| 110/110 [00:18<00:00,  5.90it/s]

2023-12-22 09:20:38,237 
Results:
- F-score (micro) 0.6191
- F-score (macro) 0.4453
- Accuracy 0.6191

By class:
              precision    recall  f1-score   support

     comment     0.9451    0.6186    0.7477      1476
     support     0.1313    0.4519    0.2035       104
        deny     0.2785    0.6600    0.3917       100
       query     0.2973    0.8333    0.4382        66

    accuracy                         0.6191      1746
   macro avg     0.4130    0.6410    0.4453      1746
weighted avg     0.8340    0.6191    0.6832      1746

2023-12-22 09:20:38,239 ----------------------------------------------------------------------------------------------------


# Train flair's model (static and character-level embeddings via GRUs)

Since the flair library allows us to easily change the underlying classifier architecture, we have also tested using a more "traditional" setup consisting of a GRU classifier (a simple Recunnet Neural Network cell architecture similar to LSTMs), where the input embedding are a concatenation of:


*   Static (GLoVe) embeddings
*   Contextual, character-level embeddings (starting from the left side)
*   Contextual, character-level embeddings (starting from the right side)

The character-level contextual embeddings are an implementation of https://aclanthology.org/C18-1139/ and predate BERT by only two months. They are implemented via a LSTM network, and can help handling out of vocabulary (OOV) words from GLoVe while also encoding a minimal amount of contextual information.

Since we have execute two RNNs for the input and another one for the output, this architecture is actually slower than a BERT model, and its performance is also slightly worse.


In [ ]:
from flair.embeddings import FlairEmbeddings, WordEmbeddings, DocumentRNNEmbeddings

In [ ]:
def test_model_configuration_character_level_embeddings(samples_train, ground_truths, add_previous_comment, add_post_title, add_depth):
  # Create the BERT samples
  bert_samples_train = create_bert_samples(samples_train, ground_truths, add_previous_comment, add_post_title, add_depth)
  bert_samples_test = create_bert_samples(samples_test, ground_truths, add_previous_comment, add_post_title, add_depth)

  print("Sample example:", bert_samples_train[42])

  # Further split train into dev+train
  random.shuffle(bert_samples_train)
  split_ratio = 0.8

  split_index = int(len(bert_samples_train) * split_ratio)

  bert_samples_dev = bert_samples_train[split_index:]
  bert_samples_train = bert_samples_train[:split_index]

  # Downsample the train dataset (the comment class is oversampled)
  bert_samples_train_downsampled = downsample(bert_samples_train)

  # Write the datasets in fasttext sample format
  to_fasttext_dataset(bert_samples_train_downsampled, "train_fasttext_format.txt")
  to_fasttext_dataset(bert_samples_dev, "dev_fasttext_format.txt")
  to_fasttext_dataset(bert_samples_test, "test_fasttext_format.txt")

  # Create flair's corpus
  data_folder = '.'
  corpus: Corpus = ClassificationCorpus(data_folder,
                                        test_file='test_fasttext_format.txt',
                                        dev_file='dev_fasttext_format.txt',
                                        train_file='train_fasttext_format.txt',
                                        label_type='class')

  print("Corpus statistics:\n", corpus.obtain_statistics())

  # Declare the embeddings we want to use
  word_embeddings = [
      # Static embeddings
      WordEmbeddings('glove'),

      # Character-level contextual embeddings, which help with adding context
      # to the static embeddings above
      FlairEmbeddings('news-forward'),
      FlairEmbeddings('news-backward'),]
  document_embeddings = DocumentRNNEmbeddings(word_embeddings)
                                              #hidden_size=512,
                                              #reproject_words=True,
                                              #reproject_words_dimension=256)

  example_sentence = bert_samples_train_downsampled[0]
  document_embeddings.embed(example_sentence)
  print(f"Example of an embedding:\n\t{example_sentence}\n\t{example_sentence.embedding}")

  # Instantiate flair's classifier (internally, it uses torch classifier layers)
  classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(label_type='class'), label_type='class')

  # And train it
  trainer = ModelTrainer(classifier, corpus)
  trainer.fine_tune('resources/taggers/rumoureval_character_level_embeddings',
                    learning_rate=5.0e-5,
                    mini_batch_size=64,
                    max_epochs=25)

  with torch.no_grad():
    torch.cuda.empty_cache()

In this case, the best performing configuration is the one that adds the previous comment's text and the comment's depth indicator.

Overall, the scores are lower than the BERT model. Since the training and inference cost is actually higher in this case, the BERT model is preferable.

Although we would need a bigger dataset to be able to establish a fair comparison between both models and come into deeper conclusions about the best configuration (we are training a whole RNN here and fine-tuning a BERT model in the previous case with very limited data), there seems to be an indication that, at the very least, adding the depth indicator helps the classifier.

In [ ]:
add_previous_comment = False
add_post_title = False
add_depth = False

test_model_configuration_character_level_embeddings(samples_train, ground_truths, add_previous_comment, add_post_title, add_depth)

Sample example: Sentence[27]: "“$MENTION$: The day #Ferguson cops told a dirty, bloody lie (via $MENTION$): $URL$ $URL$" → support (1.0)
2023-12-21 14:04:38,831 Reading data from .
2023-12-21 14:04:38,833 Train: train_fasttext_format.txt
2023-12-21 14:04:38,834 Dev: dev_fasttext_format.txt
2023-12-21 14:04:38,836 Test: test_fasttext_format.txt
2023-12-21 14:04:38,918 Initialized corpus . (label type name is 'class')
Corpus statistics:
 {
    "TRAIN": {
        "dataset": "TRAIN",
        "total_number_of_documents": 1633,
        "number_of_documents_per_class": {
            "comment": 442,
            "deny": 354,
            "support": 442,
            "query": 395
        },
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 37367,
            "min": 1,
            "max": 492,
            "avg": 22.882424984690754
        }
    },
    "TEST": {
        "dataset": "TEST",
        "total_number_of_documents": 1746,
        "number_of_documents_

100%|██████████| 153M/153M [00:09<00:00, 16.1MB/s]

2023-12-21 14:04:55,655 copying /tmp/tmpnmno8y20 to cache at /root/.flair/embeddings/glove.gensim.vectors.npy


2023-12-21 14:04:55,958 removing temp file /tmp/tmpnmno8y20
2023-12-21 14:04:56,629 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim not found in cache, downloading to /tmp/tmpp2cf6skv


100%|██████████| 20.5M/20.5M [00:02<00:00, 9.16MB/s]

...

100%|██████████| 80/80 [00:47<00:00,  1.68it/s]

2023-12-21 14:42:11,037 DEV : loss 1.2047524452209473 - f1-score (micro avg)  0.455


2023-12-21 14:42:13,168 ----------------------------------------------------------------------------------------------------
2023-12-21 14:42:13,170 Testing using last state of model ...


100%|██████████| 110/110 [00:54<00:00,  2.02it/s]

2023-12-21 14:43:07,666 
Results:
- F-score (micro) 0.4983
- F-score (macro) 0.3406
- Accuracy 0.4983

By class:
              precision    recall  f1-score   support

     comment     0.9123    0.5075    0.6522      1476
     support     0.1211    0.4519    0.1911       104
        deny     0.0685    0.2500    0.1075       100
       query     0.2849    0.7424    0.4118        66

    accuracy                         0.4983      1746
   macro avg     0.3467    0.4879    0.3406      1746
weighted avg     0.7931    0.4983    0.5844      1746

2023-12-21 14:43:07,669 ----------------------------------------------------------------------------------------------------


In [ ]:
add_previous_comment = True
add_post_title = False
add_depth = False

test_model_configuration_character_level_embeddings(samples_train, ground_truths, add_previous_comment, add_post_title, add_depth)

Sample example: Sentence[51]: "The day #Ferguson cops told a dirty, bloody lie (via $MENTION$): $URL$ $URL$ | “$MENTION$: The day #Ferguson cops told a dirty, bloody lie (via $MENTION$): $URL$ $URL$" → support (1.0)
2023-12-21 14:43:16,349 Reading data from .
2023-12-21 14:43:16,351 Train: train_fasttext_format.txt
2023-12-21 14:43:16,354 Dev: dev_fasttext_format.txt
2023-12-21 14:43:16,357 Test: test_fasttext_format.txt
2023-12-21 14:43:16,448 Initialized corpus . (label type name is 'class')
Corpus statistics:
 {
    "TRAIN": {
        "dataset": "TRAIN",
        "total_number_of_documents": 1663,
        "number_of_documents_per_class": {
            "comment": 453,
            "query": 394,
            "deny": 363,
            "support": 453
        },
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 78064,
            "min": 11,
            "max": 591,
            "avg": 46.94167167769092
        }
    },
    "TEST": {
        "dataset": "

0it [00:00, ?it/s]
1663it [00:02, 823.87it/s]

2023-12-21 14:43:38,401 Dictionary created for label 'class' with 4 values: comment (seen 453 times), support (seen 453 times), query (seen 394 times), deny (seen 363 times)
2023-12-21 14:43:38,410 ----------------------------------------------------------------------------------------------------
2023-12-21 14:43:38,414 Model: "TextClassifier(
  (embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings(
        'glove'
        (embedding): Embedding(400001, 100)
      )
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.05, inplace=False)
          (encoder): Embedding(300, 100)
          (rnn): LSTM(100, 2048)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.05, inplace=False)
          (encoder): Embedding(300, 100)
          (rnn): LSTM(100, 2048)
        )
      )
    )
    (word_reprojection_map): Linear

2023-12-21 14:43:44,944 epoch 1 - iter 2/26 - loss 1.44883072 - time (sec): 6.49 - samples/sec: 19.73 - lr: 0.000001 - momentum: 0.000000
2023-12-21 14:43:48,013 epoch 1 - iter 4/26 - loss 1.44344002 - time (sec): 9.56 - samples/sec: 26.78 - lr: 0.000002 - momentum: 0.000000
2023-12-21 14:43:52,627 epoch 1 - iter 6/26 - loss 1.43699368 - time (sec): 14.17 - samples/sec: 27.10 - lr: 0.000004 - momentum: 0.000000
2023-12-21 14:44:00,179 epoch 1 - iter 8/26 - loss 1.43938459 - time (sec): 21.72 - samples/sec: 23.57 - lr: 0.000005 - momentum: 0.000000
2023-12-21 14:44:04,167 epoch 1 - iter 10/26 - loss 1.43879520 - time (sec): 25.71 - samples/sec: 24.89 - lr: 0.000007 - momentum: 0.000000
2023-12-21 14:44:07,511 epoch 1 - iter 12/26 - loss 1.45634094 - time (sec): 29.06 - samples/sec: 26.43 - lr: 0.000008 - momentum: 0.000000
2023-12-21 14:44:13,903 epoch 1 - iter 14/26 - loss 1.45590735 - time (sec): 35.45 - samples/sec: 25.28 - lr: 0.000010 - momentum: 0.000000
2023-12-21 14:44:17,225 ep

100%|██████████| 80/80 [01:09<00:00,  1.15it/s]

...

100%|██████████| 80/80 [01:11<00:00,  1.12it/s]

2023-12-21 15:38:32,308 DEV : loss 1.1975997686386108 - f1-score (micro avg)  0.4661


2023-12-21 15:38:38,246 ----------------------------------------------------------------------------------------------------
2023-12-21 15:38:38,248 Testing using last state of model ...


100%|██████████| 110/110 [01:21<00:00,  1.34it/s]


2023-12-21 15:40:00,230 
Results:
- F-score (micro) 0.5286
- F-score (macro) 0.3357
- Accuracy 0.5286

By class:
              precision    recall  f1-score   support

     comment     0.8927    0.5522    0.6823      1476
     support     0.0975    0.3750    0.1548       104
        deny     0.0873    0.2200    0.1250       100
       query     0.2597    0.7121    0.3806        66

    accuracy                         0.5286      1746
   macro avg     0.3343    0.4648    0.3357      1746
weighted avg     0.7752    0.5286    0.6075      1746

2023-12-21 15:40:00,232 ----------------------------------------------------------------------------------------------------


In [ ]:
add_previous_comment = False
add_post_title = True
add_depth = False

test_model_configuration_character_level_embeddings(samples_train, ground_truths, add_previous_comment, add_post_title, add_depth)

Sample example: Sentence[27]: "“$MENTION$: The day #Ferguson cops told a dirty, bloody lie (via $MENTION$): $URL$ $URL$" → support (1.0)
2023-12-22 09:43:20,981 Reading data from .
2023-12-22 09:43:20,984 Train: train_fasttext_format.txt
2023-12-22 09:43:20,988 Dev: dev_fasttext_format.txt
2023-12-22 09:43:20,989 Test: test_fasttext_format.txt
2023-12-22 09:43:21,122 Initialized corpus . (label type name is 'class')
Corpus statistics:
 {
    "TRAIN": {
        "dataset": "TRAIN",
        "total_number_of_documents": 1617,
        "number_of_documents_per_class": {
            "support": 437,
            "deny": 350,
            "comment": 437,
            "query": 393
        },
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 50163,
            "min": 3,
            "max": 719,
            "avg": 31.02226345083488
        }
    },
    "TEST": {
        "dataset": "TEST",
        "total_number_of_documents": 1746,
        "number_of_documents_p

0it [00:00, ?it/s]
1617it [00:01, 1148.53it/s]

2023-12-22 09:43:37,723 Dictionary created for label 'class' with 4 values: support (seen 437 times), comment (seen 437 times), query (seen 393 times), deny (seen 350 times)
2023-12-22 09:43:37,732 ----------------------------------------------------------------------------------------------------
2023-12-22 09:43:37,737 Model: "TextClassifier(
  (embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings(
        'glove'
        (embedding): Embedding(400001, 100)
      )
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.05, inplace=False)
          (encoder): Embedding(300, 100)
          (rnn): LSTM(100, 2048)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.05, inplace=False)
          (encoder): Embedding(300, 100)
          (rnn): LSTM(100, 2048)
        )
      )
    )
    (word_reprojection_map): Linear

2023-12-22 09:43:37,764 Computation:
2023-12-22 09:43:37,765  - compute on device: cuda:0
2023-12-22 09:43:37,767  - embedding storage: none
2023-12-22 09:43:37,768 ----------------------------------------------------------------------------------------------------
2023-12-22 09:43:37,771 Model training base path: "resources/taggers/rumoureval_character_level_embeddings"
2023-12-22 09:43:37,772 ----------------------------------------------------------------------------------------------------
2023-12-22 09:43:37,773 ----------------------------------------------------------------------------------------------------
2023-12-22 09:43:41,375 epoch 1 - iter 1/13 - loss 1.51625109 - time (sec): 3.60 - samples/sec: 35.55 - lr: 0.000000 - momentum: 0.000000
2023-12-22 09:43:45,367 epoch 1 - iter 2/13 - loss 1.50705540 - time (sec): 7.59 - samples/sec: 33.72 - lr: 0.000001 - momentum: 0.000000
2023-12-22 09:43:47,815 epoch 1 - iter 3/13 - loss 1.48917218 - time (sec): 10.04 - samples/sec: 38.

100%|██████████| 80/80 [00:47<00:00,  1.68it/s]


...

2023-12-22 10:25:49,430 DEV : loss 1.2004897594451904 - f1-score (micro avg)  0.459
2023-12-22 10:25:50,570 ----------------------------------------------------------------------------------------------------
2023-12-22 10:25:54,775 epoch 25 - iter 1/13 - loss 1.02305889 - time (sec): 4.20 - samples/sec: 30.50 - lr: 0.000005 - momentum: 0.000000
2023-12-22 10:25:58,382 epoch 25 - iter 2/13 - loss 0.97893184 - time (sec): 7.80 - samples/sec: 32.81 - lr: 0.000004 - momentum: 0.000000
2023-12-22 10:26:00,955 epoch 25 - iter 3/13 - loss 0.93751264 - time (sec): 10.38 - samples/sec: 37.01 - lr: 0.000004 - momentum: 0.000000
2023-12-22 10:26:05,157 epoch 25 - iter 4/13 - loss 0.92567602 - time (sec): 14.58 - samples/sec: 35.12 - lr: 0.000004 - momentum: 0.000000
2023-12-22 10:26:13,238 epoch 25 - iter 5/13 - loss 0.92953892 - time (sec): 22.66 - samples/sec: 28.24 - lr: 0.000004 - momentum: 0.000000
2023-12-22 10:26:15,984 epoch 25 - iter 6/13 - loss 0.95873837 - time (sec): 25.41 - samples/

100%|██████████| 80/80 [00:46<00:00,  1.71it/s]


2023-12-22 10:27:33,276 DEV : loss 1.2031127214431763 - f1-score (micro avg)  0.4566
2023-12-22 10:27:39,097 ----------------------------------------------------------------------------------------------------
2023-12-22 10:27:39,099 Testing using last state of model ...


100%|██████████| 110/110 [00:56<00:00,  1.93it/s]

2023-12-22 10:28:36,038 
Results:
- F-score (micro) 0.5063
- F-score (macro) 0.3234
- Accuracy 0.5063

By class:
              precision    recall  f1-score   support

     comment     0.9022    0.5251    0.6638      1476
     support     0.0956    0.4135    0.1552       104
        deny     0.0810    0.2000    0.1153       100
       query     0.2421    0.6970    0.3594        66

    accuracy                         0.5063      1746
   macro avg     0.3302    0.4589    0.3234      1746
weighted avg     0.7822    0.5063    0.5906      1746

2023-12-22 10:28:36,039 ----------------------------------------------------------------------------------------------------


In [ ]:
add_previous_comment = True
add_post_title = True
add_depth = False

test_model_configuration_character_level_embeddings(samples_train, ground_truths, add_previous_comment, add_post_title, add_depth)

Sample example: Sentence[51]: "The day #Ferguson cops told a dirty, bloody lie (via $MENTION$): $URL$ $URL$ | “$MENTION$: The day #Ferguson cops told a dirty, bloody lie (via $MENTION$): $URL$ $URL$" → support (1.0)
2023-12-22 10:31:07,006 Reading data from .
2023-12-22 10:31:07,008 Train: train_fasttext_format.txt
2023-12-22 10:31:07,010 Dev: dev_fasttext_format.txt
2023-12-22 10:31:07,012 Test: test_fasttext_format.txt
2023-12-22 10:31:07,113 Initialized corpus . (label type name is 'class')
Corpus statistics:
 {
    "TRAIN": {
        "dataset": "TRAIN",
        "total_number_of_documents": 1591,
        "number_of_documents_per_class": {
            "support": 427,
            "comment": 427,
            "query": 395,
            "deny": 342
        },
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 88334,
            "min": 15,
            "max": 599,
            "avg": 55.52105593966059
        }
    },
    "TEST": {
        "dataset": "

0it [00:00, ?it/s]
1591it [00:01, 1188.45it/s]

2023-12-22 10:31:30,879 Dictionary created for label 'class' with 4 values: support (seen 427 times), comment (seen 427 times), query (seen 395 times), deny (seen 342 times)
2023-12-22 10:31:30,887 ----------------------------------------------------------------------------------------------------
2023-12-22 10:31:30,889 Model: "TextClassifier(
  (embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings(
        'glove'
        (embedding): Embedding(400001, 100)
      )
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.05, inplace=False)
          (encoder): Embedding(300, 100)
          (rnn): LSTM(100, 2048)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.05, inplace=False)
          (encoder): Embedding(300, 100)
          (rnn): LSTM(100, 2048)
        )
      )
    )
    (word_reprojection_map): Linear

2023-12-22 10:31:30,902  - shuffle: "True"
2023-12-22 10:31:30,903 ----------------------------------------------------------------------------------------------------
2023-12-22 10:31:30,904 Plugins:
2023-12-22 10:31:30,904  - LinearScheduler | warmup_fraction: '0.1'
2023-12-22 10:31:30,905 ----------------------------------------------------------------------------------------------------
2023-12-22 10:31:30,906 Final evaluation on model after last epoch (final-model.pt)
2023-12-22 10:31:30,907  - metric: "('micro avg', 'f1-score')"
2023-12-22 10:31:30,908 ----------------------------------------------------------------------------------------------------
2023-12-22 10:31:30,909 Computation:
2023-12-22 10:31:30,910  - compute on device: cuda:0
2023-12-22 10:31:30,911  - embedding storage: none
2023-12-22 10:31:30,912 ----------------------------------------------------------------------------------------------------
2023-12-22 10:31:30,913 Model training base path: "resources/taggers

100%|██████████| 80/80 [01:37<00:00,  1.21s/it]

...

100%|██████████| 80/80 [01:37<00:00,  1.22s/it]

2023-12-22 11:39:25,024 DEV : loss 1.2239733934402466 - f1-score (micro avg)  0.4519


2023-12-22 11:39:28,157 ----------------------------------------------------------------------------------------------------
2023-12-22 11:39:28,163 Testing using last state of model ...


100%|██████████| 110/110 [01:21<00:00,  1.34it/s]

2023-12-22 11:40:50,160 
Results:
- F-score (micro) 0.5034
- F-score (macro) 0.3337
- Accuracy 0.5034

By class:
              precision    recall  f1-score   support

     comment     0.9219    0.5115    0.6580      1476
     support     0.1072    0.5288    0.1783       104
        deny     0.1041    0.2300    0.1433       100
       query     0.2383    0.6970    0.3552        66

    accuracy                         0.5034      1746
   macro avg     0.3429    0.4918    0.3337      1746
weighted avg     0.8007    0.5034    0.5885      1746

2023-12-22 11:40:50,166 ----------------------------------------------------------------------------------------------------


In [ ]:
add_previous_comment = False
add_post_title = False
add_depth = True

test_model_configuration_character_level_embeddings(samples_train, ground_truths, add_previous_comment, add_post_title, add_depth)

Sample example: Sentence[29]: "1 | “$MENTION$: The day #Ferguson cops told a dirty, bloody lie (via $MENTION$): $URL$ $URL$" → support (1.0)
2023-12-22 11:40:54,913 Reading data from .
2023-12-22 11:40:54,914 Train: train_fasttext_format.txt
2023-12-22 11:40:54,918 Dev: dev_fasttext_format.txt
2023-12-22 11:40:54,920 Test: test_fasttext_format.txt
2023-12-22 11:40:55,008 Initialized corpus . (label type name is 'class')
Corpus statistics:
 {
    "TRAIN": {
        "dataset": "TRAIN",
        "total_number_of_documents": 1619,
        "number_of_documents_per_class": {
            "query": 392,
            "support": 438,
            "deny": 351,
            "comment": 438
        },
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 39467,
            "min": 4,
            "max": 494,
            "avg": 24.37739345274861
        }
    },
    "TEST": {
        "dataset": "TEST",
        "total_number_of_documents": 1746,
        "number_of_documen

0it [00:00, ?it/s]
1619it [00:00, 2546.84it/s]

2023-12-22 11:41:07,151 Dictionary created for label 'class' with 4 values: support (seen 438 times), comment (seen 438 times), query (seen 392 times), deny (seen 351 times)
2023-12-22 11:41:07,158 ----------------------------------------------------------------------------------------------------
2023-12-22 11:41:07,160 Model: "TextClassifier(
  (embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings(
        'glove'
        (embedding): Embedding(400001, 100)
      )
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.05, inplace=False)
          (encoder): Embedding(300, 100)
          (rnn): LSTM(100, 2048)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.05, inplace=False)
          (encoder): Embedding(300, 100)
          (rnn): LSTM(100, 2048)
        )
      )
    )
    (word_reprojection_map): Linear

2023-12-22 11:41:10,634 epoch 1 - iter 2/26 - loss 1.53695297 - time (sec): 3.44 - samples/sec: 37.19 - lr: 0.000001 - momentum: 0.000000
2023-12-22 11:41:11,484 epoch 1 - iter 4/26 - loss 1.55951932 - time (sec): 4.29 - samples/sec: 59.66 - lr: 0.000002 - momentum: 0.000000
2023-12-22 11:41:13,009 epoch 1 - iter 6/26 - loss 1.56170690 - time (sec): 5.82 - samples/sec: 66.02 - lr: 0.000004 - momentum: 0.000000
2023-12-22 11:41:14,384 epoch 1 - iter 8/26 - loss 1.54013699 - time (sec): 7.19 - samples/sec: 71.20 - lr: 0.000005 - momentum: 0.000000
2023-12-22 11:41:15,148 epoch 1 - iter 10/26 - loss 1.52234561 - time (sec): 7.95 - samples/sec: 80.46 - lr: 0.000007 - momentum: 0.000000
2023-12-22 11:41:18,083 epoch 1 - iter 12/26 - loss 1.51405248 - time (sec): 10.89 - samples/sec: 70.52 - lr: 0.000008 - momentum: 0.000000
2023-12-22 11:41:20,965 epoch 1 - iter 14/26 - loss 1.50721834 - time (sec): 13.77 - samples/sec: 65.06 - lr: 0.000010 - momentum: 0.000000
2023-12-22 11:41:24,753 epoch

100%|██████████| 80/80 [00:44<00:00,  1.79it/s]

...

100%|██████████| 80/80 [00:44<00:00,  1.79it/s]

2023-12-22 12:13:58,506 DEV : loss 1.2196269035339355 - f1-score (micro avg)  0.4543


2023-12-22 12:14:00,805 ----------------------------------------------------------------------------------------------------
2023-12-22 12:14:00,813 Testing using last state of model ...


100%|██████████| 110/110 [00:51<00:00,  2.14it/s]

2023-12-22 12:14:52,218 
Results:
- F-score (micro) 0.48
- F-score (macro) 0.3224
- Accuracy 0.48

By class:
              precision    recall  f1-score   support

     comment     0.8964    0.4925    0.6358      1476
     support     0.0886    0.4327    0.1471       104
        deny     0.0598    0.1500    0.0855       100
       query     0.2898    0.7727    0.4215        66

    accuracy                         0.4800      1746
   macro avg     0.3336    0.4620    0.3224      1746
weighted avg     0.7775    0.4800    0.5670      1746

2023-12-22 12:14:52,221 ----------------------------------------------------------------------------------------------------


In [ ]:
add_previous_comment = True
add_post_title = True
add_depth = True

test_model_configuration_character_level_embeddings(samples_train, ground_truths, add_previous_comment, add_post_title, add_depth)

Sample example: Sentence[53]: "1 | The day #Ferguson cops told a dirty, bloody lie (via $MENTION$): $URL$ $URL$ | “$MENTION$: The day #Ferguson cops told a dirty, bloody lie (via $MENTION$): $URL$ $URL$" → support (1.0)
2023-12-22 12:15:03,115 Reading data from .
2023-12-22 12:15:03,116 Train: train_fasttext_format.txt
2023-12-22 12:15:03,118 Dev: dev_fasttext_format.txt
2023-12-22 12:15:03,120 Test: test_fasttext_format.txt
2023-12-22 12:15:03,208 Initialized corpus . (label type name is 'class')
Corpus statistics:
 {
    "TRAIN": {
        "dataset": "TRAIN",
        "total_number_of_documents": 1651,
        "number_of_documents_per_class": {
            "query": 374,
            "deny": 365,
            "support": 456,
            "comment": 456
        },
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 95749,
            "min": 17,
            "max": 1139,
            "avg": 57.994548758328285
        }
    },
    "TEST": {
        "datas

0it [00:00, ?it/s]
1651it [00:02, 658.86it/s]

2023-12-22 12:15:24,868 Dictionary created for label 'class' with 4 values: support (seen 456 times), comment (seen 456 times), query (seen 374 times), deny (seen 365 times)


2023-12-22 12:15:24,876 ----------------------------------------------------------------------------------------------------
2023-12-22 12:15:24,882 Model: "TextClassifier(
  (embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings(
        'glove'
        (embedding): Embedding(400001, 100)
      )
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.05, inplace=False)
          (encoder): Embedding(300, 100)
          (rnn): LSTM(100, 2048)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.05, inplace=False)
          (encoder): Embedding(300, 100)
          (rnn): LSTM(100, 2048)
        )
      )
    )
    (word_reprojection_map): Linear(in_features=4196, out_features=4196, bias=True)
    (rnn): GRU(4196, 128, batch_first=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Linear(in_features=

100%|██████████| 80/80 [01:19<00:00,  1.00it/s]

...

100%|██████████| 80/80 [01:19<00:00,  1.00it/s]

2023-12-22 13:18:12,782 DEV : loss 1.2317458391189575 - f1-score (micro avg)  0.4574


2023-12-22 13:18:15,289 ----------------------------------------------------------------------------------------------------
2023-12-22 13:18:15,292 Testing using last state of model ...


100%|██████████| 110/110 [01:22<00:00,  1.33it/s]

2023-12-22 13:19:37,863 
Results:
- F-score (micro) 0.4914
- F-score (macro) 0.3195
- Accuracy 0.4914

By class:
              precision    recall  f1-score   support

     comment     0.9011    0.5061    0.6482      1476
     support     0.1106    0.4904    0.1805       104
        deny     0.0569    0.1600    0.0840       100
       query     0.2514    0.6667    0.3651        66

    accuracy                         0.4914      1746
   macro avg     0.3300    0.4558    0.3195      1746
weighted avg     0.7811    0.4914    0.5773      1746

2023-12-22 13:19:37,865 ----------------------------------------------------------------------------------------------------


In [ ]:
add_previous_comment = True
add_post_title = False
add_depth = True

test_model_configuration_character_level_embeddings(samples_train, ground_truths, add_previous_comment, add_post_title, add_depth)

Sample example: Sentence[53]: "1 | The day #Ferguson cops told a dirty, bloody lie (via $MENTION$): $URL$ $URL$ | “$MENTION$: The day #Ferguson cops told a dirty, bloody lie (via $MENTION$): $URL$ $URL$" → support (1.0)
2023-12-22 13:19:46,396 Reading data from .
2023-12-22 13:19:46,400 Train: train_fasttext_format.txt
2023-12-22 13:19:46,404 Dev: dev_fasttext_format.txt
2023-12-22 13:19:46,406 Test: test_fasttext_format.txt
2023-12-22 13:19:46,554 Initialized corpus . (label type name is 'class')
Corpus statistics:
 {
    "TRAIN": {
        "dataset": "TRAIN",
        "total_number_of_documents": 1624,
        "number_of_documents_per_class": {
            "support": 442,
            "comment": 442,
            "deny": 354,
            "query": 386
        },
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 78483,
            "min": 9,
            "max": 511,
            "avg": 48.32697044334975
        }
    },
    "TEST": {
        "dataset"

0it [00:00, ?it/s]
1624it [00:01, 1189.04it/s]

2023-12-22 13:20:06,987 Dictionary created for label 'class' with 4 values: support (seen 442 times), comment (seen 442 times), query (seen 386 times), deny (seen 354 times)
2023-12-22 13:20:06,996 ----------------------------------------------------------------------------------------------------
2023-12-22 13:20:06,997 Model: "TextClassifier(
  (embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings(
        'glove'
        (embedding): Embedding(400001, 100)
      )
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.05, inplace=False)
          (encoder): Embedding(300, 100)
          (rnn): LSTM(100, 2048)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.05, inplace=False)
          (encoder): Embedding(300, 100)
          (rnn): LSTM(100, 2048)
        )
      )
    )
    (word_reprojection_map): Linear

2023-12-22 13:20:10,840 epoch 1 - iter 2/26 - loss 1.48503661 - time (sec): 3.81 - samples/sec: 33.56 - lr: 0.000001 - momentum: 0.000000
2023-12-22 13:20:16,112 epoch 1 - iter 4/26 - loss 1.46680987 - time (sec): 9.09 - samples/sec: 28.17 - lr: 0.000002 - momentum: 0.000000
2023-12-22 13:20:18,750 epoch 1 - iter 6/26 - loss 1.46794643 - time (sec): 11.72 - samples/sec: 32.75 - lr: 0.000004 - momentum: 0.000000
2023-12-22 13:20:24,870 epoch 1 - iter 8/26 - loss 1.46313447 - time (sec): 17.84 - samples/sec: 28.69 - lr: 0.000005 - momentum: 0.000000
2023-12-22 13:20:29,215 epoch 1 - iter 10/26 - loss 1.46397672 - time (sec): 22.19 - samples/sec: 28.84 - lr: 0.000007 - momentum: 0.000000
2023-12-22 13:20:31,341 epoch 1 - iter 12/26 - loss 1.46504609 - time (sec): 24.32 - samples/sec: 31.59 - lr: 0.000008 - momentum: 0.000000
2023-12-22 13:20:36,953 epoch 1 - iter 14/26 - loss 1.45523047 - time (sec): 29.93 - samples/sec: 29.94 - lr: 0.000010 - momentum: 0.000000
2023-12-22 13:20:39,192 ep

100%|██████████| 80/80 [01:13<00:00,  1.09it/s]

...

2023-12-22 14:08:47,423 DEV : loss 1.1937644481658936 - f1-score (micro avg)  0.4464
2023-12-22 14:08:49,982 ----------------------------------------------------------------------------------------------------
2023-12-22 14:08:52,646 epoch 25 - iter 2/26 - loss 0.95695651 - time (sec): 2.66 - samples/sec: 48.18 - lr: 0.000003 - momentum: 0.000000
2023-12-22 14:08:57,508 epoch 25 - iter 4/26 - loss 0.90459253 - time (sec): 7.52 - samples/sec: 34.05 - lr: 0.000003 - momentum: 0.000000
2023-12-22 14:09:03,386 epoch 25 - iter 6/26 - loss 0.89393397 - time (sec): 13.40 - samples/sec: 28.66 - lr: 0.000002 - momentum: 0.000000
2023-12-22 14:09:05,756 epoch 25 - iter 8/26 - loss 0.88545369 - time (sec): 15.77 - samples/sec: 32.47 - lr: 0.000002 - momentum: 0.000000
2023-12-22 14:09:08,552 epoch 25 - iter 10/26 - loss 0.88947936 - time (sec): 18.56 - samples/sec: 34.48 - lr: 0.000002 - momentum: 0.000000
2023-12-22 14:09:14,120 epoch 25 - iter 12/26 - loss 0.88590672 - time (sec): 24.13 - sampl

100%|██████████| 80/80 [01:13<00:00,  1.09it/s]


2023-12-22 14:10:46,631 DEV : loss 1.1981905698776245 - f1-score (micro avg)  0.4495
2023-12-22 14:10:52,462 ----------------------------------------------------------------------------------------------------
2023-12-22 14:10:52,465 Testing using last state of model ...


100%|██████████| 110/110 [01:15<00:00,  1.45it/s]

2023-12-22 14:12:08,127 
Results:
- F-score (micro) 0.5344
- F-score (macro) 0.3488
- Accuracy 0.5344

By class:
              precision    recall  f1-score   support

     comment     0.9044    0.5515    0.6852      1476
     support     0.1190    0.4808    0.1908       104
        deny     0.0866    0.2200    0.1243       100
       query     0.2733    0.7121    0.3950        66

    accuracy                         0.5344      1746
   macro avg     0.3458    0.4911    0.3488      1746
weighted avg     0.7870    0.5344    0.6126      1746

2023-12-22 14:12:08,129 ----------------------------------------------------------------------------------------------------


In [ ]:
add_previous_comment = False
add_post_title = True
add_depth = True

test_model_configuration_character_level_embeddings(samples_train, ground_truths, add_previous_comment, add_post_title, add_depth)

Sample example: Sentence[29]: "1 | “$MENTION$: The day #Ferguson cops told a dirty, bloody lie (via $MENTION$): $URL$ $URL$" → support (1.0)
2023-12-22 14:12:14,208 Reading data from .
2023-12-22 14:12:14,210 Train: train_fasttext_format.txt
2023-12-22 14:12:14,212 Dev: dev_fasttext_format.txt
2023-12-22 14:12:14,215 Test: test_fasttext_format.txt
2023-12-22 14:12:14,311 Initialized corpus . (label type name is 'class')
Corpus statistics:
 {
    "TRAIN": {
        "dataset": "TRAIN",
        "total_number_of_documents": 1653,
        "number_of_documents_per_class": {
            "support": 448,
            "comment": 448,
            "deny": 359,
            "query": 398
        },
        "number_of_tokens_per_tag": {},
        "number_of_tokens": {
            "total": 54376,
            "min": 4,
            "max": 486,
            "avg": 32.895341802782816
        }
    },
    "TEST": {
        "dataset": "TEST",
        "total_number_of_documents": 1746,
        "number_of_docume

0it [00:00, ?it/s]
1653it [00:01, 1037.11it/s]

2023-12-22 14:12:32,461 Dictionary created for label 'class' with 4 values: support (seen 448 times), comment (seen 448 times), query (seen 398 times), deny (seen 359 times)
2023-12-22 14:12:32,470 ----------------------------------------------------------------------------------------------------
2023-12-22 14:12:32,472 Model: "TextClassifier(
  (embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings(
        'glove'
        (embedding): Embedding(400001, 100)
      )
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.05, inplace=False)
          (encoder): Embedding(300, 100)
          (rnn): LSTM(100, 2048)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.05, inplace=False)
          (encoder): Embedding(300, 100)
          (rnn): LSTM(100, 2048)
        )
      )
    )
    (word_reprojection_map): Linear

2023-12-22 14:12:32,502 ----------------------------------------------------------------------------------------------------
2023-12-22 14:12:32,505 Computation:
2023-12-22 14:12:32,506  - compute on device: cuda:0
2023-12-22 14:12:32,508  - embedding storage: none
2023-12-22 14:12:32,509 ----------------------------------------------------------------------------------------------------
2023-12-22 14:12:32,512 Model training base path: "resources/taggers/rumoureval_character_level_embeddings"
2023-12-22 14:12:32,513 ----------------------------------------------------------------------------------------------------
2023-12-22 14:12:32,514 ----------------------------------------------------------------------------------------------------
2023-12-22 14:12:36,047 epoch 1 - iter 2/26 - loss 1.58149987 - time (sec): 3.53 - samples/sec: 36.25 - lr: 0.000001 - momentum: 0.000000
2023-12-22 14:12:40,293 epoch 1 - iter 4/26 - loss 1.52476788 - time (sec): 7.78 - samples/sec: 32.92 - lr: 0.000

100%|██████████| 80/80 [00:50<00:00,  1.57it/s]


...

2023-12-22 14:50:30,086 DEV : loss 1.229972004890442 - f1-score (micro avg)  0.444
2023-12-22 14:50:30,764 ----------------------------------------------------------------------------------------------------
2023-12-22 14:50:36,128 epoch 25 - iter 2/26 - loss 0.81852785 - time (sec): 5.36 - samples/sec: 23.87 - lr: 0.000004 - momentum: 0.000000
2023-12-22 14:50:38,497 epoch 25 - iter 4/26 - loss 0.82659948 - time (sec): 7.73 - samples/sec: 33.11 - lr: 0.000004 - momentum: 0.000000
2023-12-22 14:50:40,653 epoch 25 - iter 6/26 - loss 0.86939549 - time (sec): 9.89 - samples/sec: 38.84 - lr: 0.000003 - momentum: 0.000000
2023-12-22 14:50:42,223 epoch 25 - iter 8/26 - loss 0.88336577 - time (sec): 11.46 - samples/sec: 44.69 - lr: 0.000003 - momentum: 0.000000
2023-12-22 14:50:45,050 epoch 25 - iter 10/26 - loss 0.85374765 - time (sec): 14.28 - samples/sec: 44.80 - lr: 0.000003 - momentum: 0.000000
2023-12-22 14:50:51,847 epoch 25 - iter 12/26 - loss 0.85352317 - time (sec): 21.08 - samples/

100%|██████████| 80/80 [00:50<00:00,  1.59it/s]


2023-12-22 14:52:03,834 DEV : loss 1.237272024154663 - f1-score (micro avg)  0.4409
2023-12-22 14:52:07,961 ----------------------------------------------------------------------------------------------------
2023-12-22 14:52:07,968 Testing using last state of model ...


100%|██████████| 110/110 [00:56<00:00,  1.93it/s]

2023-12-22 14:53:04,901 
Results:
- F-score (micro) 0.5223
- F-score (macro) 0.3334
- Accuracy 0.5223

By class:
              precision    recall  f1-score   support

     comment     0.9123    0.5427    0.6805      1476
     support     0.1086    0.4615    0.1758       104
        deny     0.0645    0.1600    0.0920       100
       query     0.2640    0.7121    0.3852        66

    accuracy                         0.5223      1746
   macro avg     0.3374    0.4691    0.3334      1746
weighted avg     0.7914    0.5223    0.6056      1746

2023-12-22 14:53:04,903 ----------------------------------------------------------------------------------------------------


# Inference with our own samples

Now that we know the best performing configuration (a DeBERTa-v3 model with a linear classifier and samples consisting of `depth | post_title | reply`), we can create our own samples in order to test its behavior on specific cases.

In [ ]:
# Download the model, saved after the last epoch (Note: this may not be available forever)
!gdown 1L9WM61gaPfZJbMWnD4SmAskEJhbU4fEq

In [27]:
sample_template = "{} | {} | {}"
classifier = TextClassifier.load('best_model_deberta.pt')

def get_prediction(post_title, reply, depth=1):
  sentence = Sentence(sample_template.format(depth, post_title, reply))
  classifier.predict(sentence)
  print(sentence.get_labels())

In [28]:
get_prediction("BREAKING NEWS! scientists confirm that microwaves cause heart problems!", "are you really sure about that?")

['Sentence[21]: "1 | BREAKING NEWS! scientists confirm that microwaves cause heart problems! | are you really sure about that?"'/'query' (0.9999)]


In [13]:
get_prediction("BREAKING NEWS! scientists confirm that microwaves cause heart problems!", "come on.. you know that's false")

['Sentence[22]: "1 | BREAKING NEWS! scientists confirm that microwaves cause heart problems! | come on.. you know that's false"'/'deny' (1.0)]


In [14]:
get_prediction("BREAKING NEWS! scientists confirm that microwaves cause heart problems!", "my uncle who works and NASA said that his third cousin's heart problems stopped when he sold his mocrowave")

['Sentence[34]: "1 | BREAKING NEWS! scientists confirm that microwaves cause heart problems! | my uncle who works and NASA said that his third cousin's heart problems stopped when he sold his mocrowave"'/'comment' (1.0)]


In [22]:
# DeBERTa's tokenizer actually supports emojis
# We can also simulate mentions (@username) and links by using the placeholders we used during training ($MENTION$ and $URL$)
get_prediction("BREAKING NEWS! scientists confirm that microwaves cause heart problems!", "$MENTION$ look at this 😂😂")

['Sentence[21]: "1 | BREAKING NEWS! scientists confirm that microwaves cause heart problems! | $MENTION$ look at this 😂😂"'/'comment' (0.9999)]


In [35]:
get_prediction("BREAKING NEWS! scientists confirm that microwaves cause heart problems!", "my best friend works at the FBI and he confirms its true!!!")

['Sentence[28]: "1 | BREAKING NEWS! scientists confirm that microwaves cause heart problems! | my best friend works at the FBI and he confirms its true!!!"'/'support' (0.9976)]
